In [9]:

import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop,Adamax,Nadam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from keras.layers.convolutional import MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

def load_preprocess_data(train_loc = 'train.npz', test_loc = 'test.npz'):
    train_ds = np.load(train_loc)
    train_x = train_ds['arr_0']
    train_y = train_ds['arr_1']

    test_ds = np.load(test_loc)
    test_x = test_ds['arr_0']

    #Normalize
    train_x = train_x/255.0
    test_x = test_x/255.0
    #Reshape
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    #one-hot encoding 
    n_classes = len(np.unique(train_y))
    train_y = np_utils.to_categorical(train_y, n_classes)

    return (train_x,train_y,test_x)

def get_model(opt):
    init_mode='he_uniform' #Weight initialization
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), kernel_initializer=init_mode,padding='same', activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1),kernel_initializer=init_mode, padding='same', activation='relu' ))
    model.add(MaxPool2D(pool_size=(2,2) ))
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1),kernel_initializer=init_mode, padding='same', activation='relu' ))
    model.add(MaxPool2D(pool_size=(1,1)))
    model.add(Conv2D(128, kernel_size=(3,3), strides=(1,1),kernel_initializer=init_mode, padding='same', activation='relu' ))
    model.add(Conv2D(128, kernel_size=(3,3), strides=(1,1),kernel_initializer=init_mode, padding='same', activation='relu' ))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(256, kernel_size=(3,3), strides=(1,1),kernel_initializer=init_mode, padding='same', activation='relu' ))
    model.add(MaxPool2D(pool_size=(1,1)))
    model.add(Conv2D(512, kernel_size=(3,3), strides=(1,1),kernel_initializer=init_mode, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(1,1)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.35))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.35))
    model.add(Dense(6, activation='softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
    return model

def train_and_predict(opt_list, train_x, train_y, test_x):
    optimizers=opt_list
    i=0
    comp={}

    result=[]
    batch_size =32
    for opti in optimizers:
        if opti=='SGD':
            epo=400
        else:
            epo=150
        callbacks = [
        ModelCheckpoint('transfer_model.h5', monitor='val_accuracy', mode='max', save_best_only=True,
                      save_weights_only=True, verbose=1),
            ] #ModelCheckpoint to save the weights of the model having the highest validation accuracy
        
        #Train-test split with changing random state and shuffle to ensure different samples of data for bagging
        X_train,X_validation,y_train,y_validation=train_test_split(train_x, train_y , test_size=0.20,random_state=i+100,shuffle=True)

        #Image Data Augmentation
        datagen = ImageDataGenerator(rotation_range=30, width_shift_range=.2, 
                                    height_shift_range=.2,vertical_flip=True,
                                    fill_mode='nearest',horizontal_flip=True,zoom_range=0.1,shear_range = 0.2 ) 
        datagen.fit(X_train)

        #Getting the model
        model = get_model(opti)
#         print(model.summary())

        #training the model
        history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                                epochs=epo,
                                validation_data=(X_validation, y_validation),callbacks=callbacks 
                                    )
        print('Maximum validation accuracy',opti, max(history.history['val_accuracy']))
        comp[str(opti)+str(i)]=max(history.history['val_accuracy'])

        #predicting on the model
        result.append(model.predict(test_x))

        i=i+1
    print(comp)
    return result

def get_final_result(result, num_models):
    nets = num_models 
    for i in range(0,nets):
        if i==0:
            y_pred=result[i]
        else:
            y_pred += result[i]
    y_pred = y_pred/nets #Average predicted class probabilities from base learners
    pred_label = np.argmax(y_pred, axis=-1)
    i=list(range(0,len(pred_label)))
    my_submission = pd.DataFrame({'Id': i, 'Category': pred_label})
    my_submission.to_csv('submission5.csv', index=False)
    return pred_label

def main():
    opt_list = ['nadam','nadam','nadam','nadam','adamax','adamax','adamax','adamax','SGD','SGD','SGD','SGD',Adam(decay=1e-2/175),Adam(decay=1e-2/175),Adam(decay=1e-2/175),Adam(decay=1e-2/175)]
    train_x,train_y,test_x = load_preprocess_data(train_loc = '/kaggle/input/ift3395-6390-quickdraw/train.npz',
                                                  test_loc = '/kaggle/input/ift3395-6390-quickdraw/test.npz')
    result = train_and_predict(opt_list, train_x, train_y, test_x)
    preds = get_final_result(result, len(opt_list))
    return preds


if __name__ == "__main__":
    main()

Epoch 1/150
37/38 [============================>.] - ETA: 0s - loss: 2.0968 - accuracy: 0.2010
Epoch 00001: val_accuracy improved from -inf to 0.18667, saving model to transfer_model.h5
38/38 [==============================] - 1s 33ms/step - loss: 2.0925 - accuracy: 0.2008 - val_loss: 1.7551 - val_accuracy: 0.1867
Epoch 2/150
37/38 [============================>.] - ETA: 0s - loss: 1.7703 - accuracy: 0.2098
Epoch 00002: val_accuracy improved from 0.18667 to 0.34333, saving model to transfer_model.h5
38/38 [==============================] - 1s 37ms/step - loss: 1.7693 - accuracy: 0.2133 - val_loss: 1.6985 - val_accuracy: 0.3433
Epoch 3/150
36/38 [===========================>..] - ETA: 0s - loss: 1.7355 - accuracy: 0.2667
Epoch 00003: val_accuracy improved from 0.34333 to 0.35000, saving model to transfer_model.h5
38/38 [==============================] - 1s 23ms/step - loss: 1.7352 - accuracy: 0.2642 - val_loss: 1.5905 - val_accuracy: 0.3500
Epoch 4/150
37/38 [===========================

Epoch 29/150
35/38 [==========================>...] - ETA: 0s - loss: 0.9440 - accuracy: 0.6404
Epoch 00029: val_accuracy did not improve from 0.73000
38/38 [==============================] - 1s 17ms/step - loss: 0.9672 - accuracy: 0.6350 - val_loss: 0.8447 - val_accuracy: 0.7067
Epoch 30/150
37/38 [============================>.] - ETA: 0s - loss: 0.9618 - accuracy: 0.6421
Epoch 00030: val_accuracy did not improve from 0.73000
38/38 [==============================] - 1s 18ms/step - loss: 0.9575 - accuracy: 0.6458 - val_loss: 0.7320 - val_accuracy: 0.7300
Epoch 31/150
35/38 [==========================>...] - ETA: 0s - loss: 1.0038 - accuracy: 0.6404
Epoch 00031: val_accuracy improved from 0.73000 to 0.74000, saving model to transfer_model.h5
38/38 [==============================] - 1s 26ms/step - loss: 1.0004 - accuracy: 0.6400 - val_loss: 0.7478 - val_accuracy: 0.7400
Epoch 32/150
36/38 [===========================>..] - ETA: 0s - loss: 0.9545 - accuracy: 0.6717
Epoch 00032: val_accur

35/38 [==========================>...] - ETA: 0s - loss: 0.6569 - accuracy: 0.7600
Epoch 00057: val_accuracy did not improve from 0.82333
38/38 [==============================] - 1s 17ms/step - loss: 0.6488 - accuracy: 0.7617 - val_loss: 0.7402 - val_accuracy: 0.7700
Epoch 58/150
35/38 [==========================>...] - ETA: 0s - loss: 0.6666 - accuracy: 0.7844
Epoch 00058: val_accuracy did not improve from 0.82333
38/38 [==============================] - 1s 17ms/step - loss: 0.6708 - accuracy: 0.7800 - val_loss: 0.6042 - val_accuracy: 0.8033
Epoch 59/150
37/38 [============================>.] - ETA: 0s - loss: 0.6802 - accuracy: 0.7748
Epoch 00059: val_accuracy did not improve from 0.82333
38/38 [==============================] - 1s 25ms/step - loss: 0.6765 - accuracy: 0.7742 - val_loss: 0.5324 - val_accuracy: 0.8067
Epoch 60/150
37/38 [============================>.] - ETA: 0s - loss: 0.6985 - accuracy: 0.7543
Epoch 00060: val_accuracy did not improve from 0.82333
38/38 [============

Epoch 86/150
35/38 [==========================>...] - ETA: 0s - loss: 0.4842 - accuracy: 0.8261
Epoch 00086: val_accuracy did not improve from 0.84333
38/38 [==============================] - 1s 17ms/step - loss: 0.5011 - accuracy: 0.8242 - val_loss: 0.4838 - val_accuracy: 0.8300
Epoch 87/150
37/38 [============================>.] - ETA: 0s - loss: 0.5435 - accuracy: 0.8202
Epoch 00087: val_accuracy did not improve from 0.84333
38/38 [==============================] - 1s 18ms/step - loss: 0.5405 - accuracy: 0.8208 - val_loss: 0.5666 - val_accuracy: 0.8267
Epoch 88/150
38/38 [==============================] - ETA: 0s - loss: 0.5286 - accuracy: 0.8175
Epoch 00088: val_accuracy did not improve from 0.84333
38/38 [==============================] - 1s 18ms/step - loss: 0.5286 - accuracy: 0.8175 - val_loss: 0.4842 - val_accuracy: 0.8300
Epoch 89/150
38/38 [==============================] - ETA: 0s - loss: 0.5235 - accuracy: 0.8217
Epoch 00089: val_accuracy did not improve from 0.84333
38/38 

Epoch 115/150
37/38 [============================>.] - ETA: 0s - loss: 0.4211 - accuracy: 0.8433
Epoch 00115: val_accuracy did not improve from 0.86667
38/38 [==============================] - 1s 18ms/step - loss: 0.4194 - accuracy: 0.8450 - val_loss: 0.5378 - val_accuracy: 0.8367
Epoch 116/150
36/38 [===========================>..] - ETA: 0s - loss: 0.4227 - accuracy: 0.8486
Epoch 00116: val_accuracy did not improve from 0.86667
38/38 [==============================] - 1s 18ms/step - loss: 0.4395 - accuracy: 0.8450 - val_loss: 0.4924 - val_accuracy: 0.8467
Epoch 117/150
37/38 [============================>.] - ETA: 0s - loss: 0.4264 - accuracy: 0.8656
Epoch 00117: val_accuracy did not improve from 0.86667
38/38 [==============================] - 1s 19ms/step - loss: 0.4300 - accuracy: 0.8650 - val_loss: 0.5418 - val_accuracy: 0.8433
Epoch 118/150
38/38 [==============================] - ETA: 0s - loss: 0.4348 - accuracy: 0.8458
Epoch 00118: val_accuracy did not improve from 0.86667
38

...

35/38 [==========================>...] - ETA: 0s - loss: 1.2723 - accuracy: 0.5226
Epoch 00022: val_accuracy did not improve from 0.65000
38/38 [==============================] - 1s 17ms/step - loss: 1.2635 - accuracy: 0.5258 - val_loss: 1.1293 - val_accuracy: 0.6000
Epoch 23/150
38/38 [==============================] - ETA: 0s - loss: 1.2321 - accuracy: 0.5625
Epoch 00023: val_accuracy did not improve from 0.65000
38/38 [==============================] - 1s 17ms/step - loss: 1.2321 - accuracy: 0.5625 - val_loss: 1.0548 - val_accuracy: 0.6167
Epoch 24/150
36/38 [===========================>..] - ETA: 0s - loss: 1.1974 - accuracy: 0.5431
Epoch 00024: val_accuracy did not improve from 0.65000
38/38 [==============================] - 1s 16ms/step - loss: 1.1954 - accuracy: 0.5483 - val_loss: 1.1356 - val_accuracy: 0.5867
Epoch 25/150
37/38 [============================>.] - ETA: 0s - loss: 1.1008 - accuracy: 0.6036
Epoch 00025: val_accuracy improved from 0.65000 to 0.65333, saving model t

Epoch 51/150
35/38 [==========================>...] - ETA: 0s - loss: 0.7566 - accuracy: 0.7292
Epoch 00051: val_accuracy improved from 0.78333 to 0.78667, saving model to transfer_model.h5
38/38 [==============================] - 1s 24ms/step - loss: 0.7427 - accuracy: 0.7325 - val_loss: 0.6272 - val_accuracy: 0.7867
Epoch 52/150
38/38 [==============================] - ETA: 0s - loss: 0.7586 - accuracy: 0.7367
Epoch 00052: val_accuracy improved from 0.78667 to 0.80667, saving model to transfer_model.h5
38/38 [==============================] - 1s 23ms/step - loss: 0.7586 - accuracy: 0.7367 - val_loss: 0.5724 - val_accuracy: 0.8067
Epoch 53/150
38/38 [==============================] - ETA: 0s - loss: 0.6689 - accuracy: 0.7633
Epoch 00053: val_accuracy did not improve from 0.80667
38/38 [==============================] - 1s 22ms/step - loss: 0.6689 - accuracy: 0.7633 - val_loss: 0.6558 - val_accuracy: 0.7733
Epoch 54/150
36/38 [===========================>..] - ETA: 0s - loss: 0.7179 - 

Epoch 80/150
36/38 [===========================>..] - ETA: 0s - loss: 0.5715 - accuracy: 0.7958
Epoch 00080: val_accuracy improved from 0.85333 to 0.85667, saving model to transfer_model.h5
38/38 [==============================] - 1s 23ms/step - loss: 0.5565 - accuracy: 0.8008 - val_loss: 0.6245 - val_accuracy: 0.8567
Epoch 81/150
37/38 [============================>.] - ETA: 0s - loss: 0.5279 - accuracy: 0.8082
Epoch 00081: val_accuracy did not improve from 0.85667
38/38 [==============================] - 1s 16ms/step - loss: 0.5286 - accuracy: 0.8075 - val_loss: 0.6135 - val_accuracy: 0.8300
Epoch 82/150
37/38 [============================>.] - ETA: 0s - loss: 0.5849 - accuracy: 0.7971
Epoch 00082: val_accuracy did not improve from 0.85667
38/38 [==============================] - 1s 17ms/step - loss: 0.5837 - accuracy: 0.7967 - val_loss: 0.6258 - val_accuracy: 0.8333
Epoch 83/150
37/38 [============================>.] - ETA: 0s - loss: 0.5523 - accuracy: 0.8074
Epoch 00083: val_accur

Epoch 109/150
37/38 [============================>.] - ETA: 0s - loss: 0.4160 - accuracy: 0.8587
Epoch 00109: val_accuracy did not improve from 0.86000
38/38 [==============================] - 1s 17ms/step - loss: 0.4155 - accuracy: 0.8575 - val_loss: 0.5910 - val_accuracy: 0.8200
Epoch 110/150
36/38 [===========================>..] - ETA: 0s - loss: 0.4933 - accuracy: 0.8266
Epoch 00110: val_accuracy did not improve from 0.86000
38/38 [==============================] - 1s 17ms/step - loss: 0.4929 - accuracy: 0.8267 - val_loss: 0.6201 - val_accuracy: 0.8267
Epoch 111/150
36/38 [===========================>..] - ETA: 0s - loss: 0.4622 - accuracy: 0.8583
Epoch 00111: val_accuracy did not improve from 0.86000
38/38 [==============================] - 1s 16ms/step - loss: 0.4700 - accuracy: 0.8567 - val_loss: 0.5500 - val_accuracy: 0.8533
Epoch 112/150
37/38 [============================>.] - ETA: 0s - loss: 0.4384 - accuracy: 0.8485
Epoch 00112: val_accuracy did not improve from 0.86000
38

Epoch 138/150
37/38 [============================>.] - ETA: 0s - loss: 0.3945 - accuracy: 0.8502
Epoch 00138: val_accuracy did not improve from 0.88333
38/38 [==============================] - 1s 16ms/step - loss: 0.3905 - accuracy: 0.8525 - val_loss: 0.6535 - val_accuracy: 0.8167
Epoch 139/150
35/38 [==========================>...] - ETA: 0s - loss: 0.3880 - accuracy: 0.8596
Epoch 00139: val_accuracy did not improve from 0.88333
38/38 [==============================] - 1s 16ms/step - loss: 0.3861 - accuracy: 0.8592 - val_loss: 0.5550 - val_accuracy: 0.8633
Epoch 140/150
36/38 [===========================>..] - ETA: 0s - loss: 0.3642 - accuracy: 0.8706
Epoch 00140: val_accuracy did not improve from 0.88333
38/38 [==============================] - 1s 16ms/step - loss: 0.3660 - accuracy: 0.8717 - val_loss: 0.6079 - val_accuracy: 0.8500
Epoch 141/150
36/38 [===========================>..] - ETA: 0s - loss: 0.4036 - accuracy: 0.8671
Epoch 00141: val_accuracy did not improve from 0.88333
38